In [1]:
%matplotlib widget

In [2]:
from functools import partial

In [3]:
from libschrodinger.crank_nicolson_2d import *

In [4]:
from libschrodinger.potentials import *

In [5]:
import numpy as np

In [6]:
import pandas as pd

In [7]:
from pathlib import Path

In [8]:
caseName : str = "tunnelSteppingTest7"

In [9]:
math = np
spatialStep : float = .05
length : float = 20
temporalStep : float = (spatialStep ** 2) / 4
courantNumber : float = 1.0
pointCount : int = int(length / spatialStep)
potentialHeight : float = 400
print("Point Count: ", pointCount)
packetStandardDeviation : float = .5
packetStartX : float = 1 - (3 * (packetStandardDeviation / length))
packetStartY : float = .5
packetWaveNumber : float = 13 * np.pi
barrierCount : int = 10
postRegionLength = .3
preRegionLength : float = ((packetStandardDeviation / length) * 12)
intraRegionLength : float = 1 - (postRegionLength + preRegionLength)
barrierWidth : float = intraRegionLength / barrierCount
constantPotentialLengthRatios : List[float] = [postRegionLength] \
        + ([barrierWidth] * barrierCount) \
        + [preRegionLength]
print(intraRegionLength)
print(postRegionLength)
print(preRegionLength)
print(barrierWidth)
print(intraRegionLength + postRegionLength + preRegionLength)
print(np.sum(constantPotentialLengthRatios ))
frameCount : int = 650
backend = "numpy"

Point Count:  400
0.3999999999999999
0.3
0.30000000000000004
0.039999999999999994
1.0
1.0


In [10]:
def recordTotalLengthWiseProbabilities(
            simulator : Simulator, 
            regionLengths : List[float], 
            regionLabels : Tuple[str], 
            math = np, 
            regionLabelPrepend = "TotalProbability::"
        ) -> pd.DataFrame:
    assert len(regionLabels) == len(regionLengths)
    currentPosition : float = 0
    regionalProbabilities : List[np.array] = []
    regionalCutFrames : List[np.array] = []
    for ii in range(len(regionLengths)): 
        regionLength = regionLengths[ii]
        probabilities, cutFrames = totalProbabilityInRegion(
                simulator.probabilities, 
                simulator.grid.pointCount, 
                simulator.spaceStep, 
                currentPosition, 
                0, 
                regionLength, 
                1, 
                math
            )
        regionalProbabilities.append(probabilities)
        regionalCutFrames.append(cutFrames)
        currentPosition += regionLength
    totalProbabilities = {
            (regionLabelPrepend + "Probabilities::" + regionLabels[ii]) : regionalProbabilities[ii] \
            for ii in range(len(regionLengths))
        }
    return totalProbabilities, regionalCutFrames

In [11]:
def performAndLogConstantSimulations(
            baseName : str, 
            frameCount : int, 
            length : float, 
            simulationProfiles : List[SimulationProfile], 
            constantRegionLengths : List[float], 
            constantRegionLabels : List[str], 
            showBar : bool = False, 
            showFPS : bool = False, 
            showTotalTime : bool = False, 
            showWhenSimulationDone : bool = False, 
            math = np, 
            basePath = None, 
            animationInterval = 30
        ):
    simulationCount = 0
    allData = {}
    basePath = basePath if basePath else Path.cwd() / baseName
    for profile in simulationProfiles: 
        simulator = Simulator(profile)
        simulator.simulate(frameCount, showBar, showFPS, showTotalTime)
        if showWhenSimulationDone == True: 
            print("Simulation " + str(simulationCount) + " is done, processing probabilities.")
        probabilities, probabilityDecibles = simulator.processProbabilities()
        if showWhenSimulationDone == True: 
            print("Simulation " + str(simulationCount) + ": done processing probabilities.")
        if showWhenSimulationDone == True: 
            print("Simulation " + str(simulationCount) + ": logging.")
        name = baseName + str(simulationCount) + "::"
        if showWhenSimulationDone == True: 
            print("Saving Video of " + name[:-2])
        videoPath = basePath / str(simulationCount)
        videoPath.mkdir(parents = True, exist_ok = True)
        totalProbabilities, cutFrames = recordTotalLengthWiseProbabilities(
                simulator, 
                constantRegionLengths, 
                constantRegionLabels, 
                math, 
                name
            )
        waveAnimation = animateImages(
                length, 
                simulator.probabilities, 
                animationInterval, 
                0, 
                math.max(simulator.probabilities), 
                constantRegionLengths, 
                [1] * len(constantRegionLengths), 
                colorMap = "hot"
            )
        waveAnimation.save(str(videoPath / (str(simulationCount) + ".mp4")))
        plt.close()
        plt.figure()
        plt.imshow(simulator.potentials[0])
        plt.savefig(str(videoPath / (str(simulationCount) + "Potential.png")))
        plt.close()
        totalProbabilities[name + "FrameCount"] = frameCount
        totalProbabilities[name + "SpaceStep"] = profile.spaceStep
        totalProbabilities[name + "TimeStep"] = profile.timeStep
        totalProbabilities[name + "PointCount"] = profile.grid.pointCount
        for ii in range(len(constantRegionLabels)): 
            print("Saving Video of " + constantRegionLabels[ii])
            totalProbabilities[name + "RegionLength::" + constantRegionLabels[ii]] = constantRegionLengths[ii]
            cutAnimation = animateImages(
                    length * constantRegionLengths[ii], 
                    cutFrames[ii], 
                    animationInterval, 
                    0, 
                    math.max(cutFrames[ii]), 
                    colorMap = "hot"
                )
            cutAnimation.save(str(videoPath / (constantRegionLabels[ii] + ".mp4")))
            plt.close()
        allData |= totalProbabilities
        if showWhenSimulationDone == True: 
            print("Done logging " + name[:-2])
        simulationCount += 1
    if showWhenSimulationDone == True: 
        print("Producing Simulation final output CSV")
    pd.DataFrame(allData).to_csv(str(basePath / (baseName + ".csv")))
    if showWhenSimulationDone == True: 
        print("Done.")

In [12]:
profiles : List[SimulationProfile] = []
potentialFunction = lambda potentialRatios, position, time : constantPotentials(
            position, 
            math.array(constantPotentialLengthRatios), 
            potentialRatios, 
            potentialHeight, 
            math
    )

potentialRatios = [0.0 for ii in range(barrierCount + 2)]
profile = SimulationProfile(
        makeLinspaceGrid(pointCount, length, 2, False, float, math), 
        lambda position : makeWavePacket(
                position, 
                packetStartX * length, 
                packetStartY * length, 
                spatialStep, 
                packetStandardDeviation, 
                packetWaveNumber, 
                math
            ), 
        partial(potentialFunction, potentialRatios), 
        temporalStep, 
        spatialStep, 
        False, 
        True, 
        False, 
        constantPotential = True
    )
profiles.append(profile) # Control Free Particle

for ii in range(barrierCount): 
    potentialRatios = math.array([0] + [1.0 if jj == ii else 0.0 \
            for jj in range(barrierCount)
        ] + [0])
    profile = SimulationProfile(
        makeLinspaceGrid(pointCount, length, 2, False, float, math), 
        lambda position : makeWavePacket(
                position, 
                packetStartX * length, 
                packetStartY * length, 
                spatialStep, 
                packetStandardDeviation, 
                packetWaveNumber, 
                math
            ), 
        partial(potentialFunction, potentialRatios), 
        temporalStep, 
        spatialStep, 
        False, 
        True, 
        False, 
        constantPotential = True
    )
    profiles.append(profile)


In [13]:
performAndLogConstantSimulations(
        caseName, 
        frameCount, 
        length, 
        profiles, 
        [postRegionLength, intraRegionLength, preRegionLength], 
        ["PostRegion", "IntraRegion", "PreRegion"], 
        True, 
        True, 
        True, 
        True, 
        math
    )

[====================================================================================================]
[-

KeyboardInterrupt: 